In [2]:
from sklearn import svm, metrics
from skimage.feature import local_binary_pattern
import numpy as np
import cv2
from os import listdir
from os.path import isfile, join

# Definir la ruta de la carpeta con las imágenes
path = 'raw'

# Categorías
categorias = ['NORMAL', 'PNEUMONIA']

# Parámetros para LBP
radio = 3
n_puntos = 8 * radio

def cargar_datos(ruta, size=(128, 128)):
    datos = []
    etiquetas = []
    for i, cat in enumerate(categorias):
        carpeta = join(ruta, cat)
        archivos = [f for f in listdir(carpeta) if isfile(join(carpeta, f))]
        for archivo in archivos:
            imagen_path = join(carpeta, archivo)
            imagen = cv2.imread(imagen_path, cv2.IMREAD_GRAYSCALE)
            if imagen is None:
                print(f'No se pudo leer la imagen: {imagen_path}')
                continue

            # Redimensionar imagen
            imagen = cv2.resize(imagen, size)

            # Calcular LBP
            lbp = local_binary_pattern(imagen, n_puntos, radio, method="uniform")
            (hist, _) = np.histogram(lbp.ravel(),
                                      bins=np.arange(0, n_puntos + 3),
                                      range=(0, n_puntos + 2))

            # Normalizar el histograma
            hist = hist.astype("float")
            hist /= (hist.sum() + 1e-7)

            datos.append(hist)
            etiquetas.append(i)

    if len(datos) == 0 or len(etiquetas) == 0:
        raise ValueError("No se pudieron cargar los datos. Asegúrate de que la ruta y las categorías son correctas.")

    return np.array(datos), np.array(etiquetas)

    # Cargar datos de entrenamiento, validación y prueba
datos_entrenamiento, etiquetas_entrenamiento = cargar_datos(join(path, 'train'))
datos_prueba, etiquetas_prueba = cargar_datos(join(path, 'test'))

print(etiquetas_entrenamiento)
print(etiquetas_prueba)

# Asegurarse de que hay datos para al menos dos clases
if len(np.unique(etiquetas_entrenamiento)) < 2 or len(np.unique(etiquetas_prueba)) < 2:
    raise ValueError("Se debe cargar datos para al menos dos clases en cada conjunto de datos.")

# Crear y entrenar el modelo SVM
modelo = svm.SVC(gamma='scale')
modelo.fit(datos_entrenamiento, etiquetas_entrenamiento)

# Evaluar el modelo
etiquetas_predichas = modelo.predict(datos_prueba)
precision = metrics.accuracy_score(etiquetas_prueba, etiquetas_predichas) * 100
print(f'Precisión del modelo: {precision:.2f}%')

[0 0 0 ... 1 1 1]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 